# Pedestrian street networks for Australian Captial cities

Using Openstreetmap as a source for pedestrian street networks derived using osmnx by way of a custom query (like OSMnx 'walk' type, but not excluding cycleways).  The 10km buffered Greater Capital City areas (according to ABS 2016 definition) in EPSG 4326 WGS84 projection are used as a polygon boundary for the network areas.

First, the network for Melbourne is calculated as a proof of concept; then, the process is generalised to iterate over a list of city names contained in the boundary shape file names.

Processing of Melbourne took approximately 40 minutes; so, if all goes well it is envisaged that the 7 city loop will be complete in under 4 hours (noting that Melbourne is a particularly large and complex network).

In [1]:
import networkx as nx
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import fiona
ox.config(use_cache=True, log_console=True)
ox.__version__

from shapely.geometry import shape 

Here are the options for network types, and what they mean in terms of edge definitions:

• drive: get drivable public streets (but not service roads)
• drive_service: get drivable public streets including service roads
• walk: get all streets and paths that pedestrians can use (this network
type ignores one-way directionality by always connecting adjacent
nodes with reciprocal directed edges)
• bike: get all streets and paths that cyclists can use
• all: download all (non-private) OpenStreetMap streets and paths
• all_private: download all OpenStreetMap streets and paths, including
private-access

In this notebook we plan to get the Walk and Bike networks for the Melbourne ABS 2016 GCCSA area, and combine these for use as a pedestrian walkable network.  In the Melbourne context, it is considered that a pedestrian may go anywhere that a cyclist might go (the reverse does not hold).

In [2]:
# get GCCSA shape file as polygon
c = fiona.open('D:/ntnl_li_2018_template/data/study_region/melb_gccsa_2016_10000m.shp')
pol = next(iter(c))
geom = shape(pol['geometry'])

## Get networks and save as graphs - Melbourne

In [8]:
# retrieve walking network bounded by GCCSA
pedestrian = '["area"!~"yes"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["service"!~"private"]["access"!~"private"]'

W = ox.graph_from_polygon(geom, custom_filter= pedestrian)
# fig, ax = ox.plot_graph(W)

# save graph to disk as shapefile 
ox.save_graph_shapefile(W, filename='osm_gccsa10km_melb_pedestrian_20180628')

## Get networks and save as graphs - Other cities

In [9]:
# define pedestrian network custom filter (based on OSMnx 'walk' network type, without the cycling exclusion)
pedestrian = '["area"!~"yes"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["service"!~"private"]["access"!~"private"]'

# retrieve walking network bounded by GCCSA
cities = ['adelaide','bris','canberra','darwin','hobart','perth','syd']
geom = []
for city in cities:
    c = fiona.open('D:/ntnl_li_2018_template/data/study_region/wgs84_epsg4326/{}_gccsa_2016_10000m.shp'.format(city))
    pol = next(iter(c))
    geom.append(shape(pol['geometry']))

In [10]:
for city in cities:
    # Retrieve and save walking network bounded by GCCSA
    W = ox.graph_from_polygon(geom[cities.index(city)],  custom_filter= pedestrian)
    ox.save_graph_shapefile(W, filename='osm_gccsa10km_{}_pedestrian_20180628'.format(city))   